In [2]:
import sqlite3 as lite
import pandas as pd

In [4]:
df_ = pd.read_excel('../Data/BTech22 Data.xlsx')

df_ = df_.fillna('unknown')
df = df_[df_.Sex != 'unknown']
df.columns

Index(['USN', 'Name', 'Email', 'Section', 'Sex', 'DoB', 'Height_cm',
       'Weight_Kg', 'Home_State', 'Home_Town', 'Mother_Tongue', 'Elective',
       'Clubs'],
      dtype='object')

In [5]:
con = lite.connect(":memory:") 

In [6]:
sql = """
CREATE TABLE Students(
   'USN' TEXT PRIMARY KEY, 
   'Name' TEXT, 
   'Email' TEXT UNIQUE, 
   'Section' TEXT, 
   'Sex' TEXT, 
   'DoB' TEXT, 
   'Height_cm' INT,
   'Weight_Kg' INT, 
   'Home_State' TEXT, 
   'Home_Town' TEXT, 
   'Mother_Tongue' TEXT, 
   'Elective' TEXT,
   'Clubs' TEXT
)
"""
con.execute(sql)
con.commit()

In [7]:
pd.read_sql("SELECT * FROM sqlite_master", con)[['type','name','tbl_name']]

,type,name,tbl_name
0,table,Students,Students
1,index,sqlite_autoindex_Students_1,Students
2,index,sqlite_autoindex_Students_2,Students


#### SQLite automatically creates two types of indexes:  

1. Internal indexes are created for all **UNIQUE** and **PRIMARY KEY** constraints and permanently added to the DB.   
2. Temporary indexes are created on a per-query basis,  
    if it is faster to create the index in RAM first and use it to speed up the original query.  


In [8]:
df.to_sql('Students', con, if_exists='append', index=False)
con.commit()

##### Distinct values in `Sex` field

In [15]:
sql = """
SELECT DISTINCT Sex FROM Students
"""
pd.read_sql(sql, con)

,Sex
0,F
1,M


##### Unify the data  
1. remove leadning and trailing spaces   
2. convert the contents of the field to UPPER CASE
3. set the first character ('F' or 'M') as the value  

In [14]:
# https://www.sqlitetutorial.net/sqlite-string-functions/

sql1 = """
    UPDATE Students 
    SET Sex = TRIM(Sex)    
"""

sql2 = """
    UPDATE Students 
    SET
     Sex = UPPER(Sex);
"""

sql3 = """
    UPDATE Students 
    SET
     Sex = SUBSTR(Sex, 1, 1)
"""

# con.execute(sql1)   # 10 to 6 variations  
# con.execute(sql2)   #  6 to 3 variations
con.execute(sql3)   #  3 to 2 variations 
con.commit()
 

#### Munge State Names   

In [16]:
sql = """
SELECT DISTINCT Home_State FROM Students
"""
pd.read_sql(sql, con)

,Home_State
0,Karnataka
1,unknown
2,Karnataka
3,Jammu
4,Haryana
5,karnataka
6,Delhi
7,karnataka
8,West Bengal
9,Andhra Pradesh


In [32]:
# ['Karnataka','karnataka','KARNATAKA','karntaka','KARnataka','Karantaka')

sql1 = """
    UPDATE Students 
    SET Home_State = TRIM(Home_State)    
"""

sql2 = """
    UPDATE Students 
    SET Home_State = 'Karnataka'
    WHERE Home_State IN 
    ('karnataka','KARNATAKA','karntaka','KARnataka','Karantaka')
"""

sql3 = """
    UPDATE Students 
    SET Home_State = 'unknown'
    WHERE Home_State = 'Nil'
"""

# con.execute(sql1)     # 28 to 22
# con.execute(sql2)     # 22 to 17
# con.execute(sql3)     # 17 to 16
con.commit()
 

##### Task   
Cleanse other categorical data in the table.    
Create look-up tables (aka master tables) for states, towns, and languages.  
See [RTO Codes](https://www.v3cars.com/car-guide/state-code-regional-transport-office-rto-india) and 
Census Codes for [States and Cities](https://censusindia.gov.in/nada/index.php/catalog/42648) and 
[Language Codes](https://censusindia.gov.in/nada/index.php/catalog/42561)  
Use standard codes or abbreviations as PK.  

Create tables for electives and clubs.  
Their PK may be auto-incremented.  